In [3]:
import pandas as pd
import polars as pl



In [13]:
# carregar input no endereço URI: c:\Users\andre\Documents\scripts python\testebtc.parquet
df = pd.read_parquet(r'..\03_output\BTC_OFICIAL.parquet')

# Selecionar Colunas: 'DATA', 'close','moeda' e  'mayer'
df = df[['DATA', 'close', 'moeda', 'mayer']]

df = df.copy()

# ordenar data  (ascending)
df = df.sort_values(['DATA'])

# Primeiro, certifique-se de que a coluna 'DATA' é do tipo datetime
df['DATA'] = pd.to_datetime(df['DATA'])

In [14]:
# Crie função para o sinal de compra, venda ou mantem ativo
def avaliar_mayer(row):
    if row['mayer'] < 1:
        return 'comprar'
    elif row['mayer'] > 1:
        return 'vender'
    else:
        return 'manter'

# Crie uma nova coluna 'acao' aplicando a função 'avaliar_mayer' a cada linha
df['acao'] = df.apply(avaliar_mayer, axis=1)


In [15]:
# Inicialize a coluna 'janela' com 0
df['janela'] = 0

# Inicialize um contador para as janelas
contador_janela = 0

# Atualize a coluna 'janela' com base na coluna 'acao'
for i in range(1, len(df)):
    if df.iloc[i]['acao'] == 'comprar' and df.iloc[i-1]['acao'] != 'comprar':
        contador_janela += 1  # Começa uma nova janela
    df.iloc[i, df.columns.get_loc('janela')] = contador_janela

df

,DATA,close,moeda,mayer,acao,janela
1510,2020-02-02,9338.20,BTCUSDT,NaN,manter,0
1509,2020-02-03,9300.64,BTCUSDT,NaN,manter,0
1508,2020-02-04,9198.35,BTCUSDT,NaN,manter,0
1507,2020-02-05,9630.00,BTCUSDT,NaN,manter,0
1506,2020-02-06,9778.12,BTCUSDT,NaN,manter,0
...,...,...,...,...,...,...
4,2024-03-18,67660.00,BTCUSDT,1.666041,vender,12
3,2024-03-19,61974.10,BTCUSDT,1.519266,vender,12
2,2024-03-20,67876.40,BTCUSDT,1.655431,vender,12
1,2024-03-21,65539.90,BTCUSDT,1.590769,vender,12


In [16]:
# Definindo constante
APORTE = 100

# Identificando o último dia de cada janela
df['ultimo_dia_janela'] = df['janela'] != df['janela'].shift(-1)

# Inicializando as colunas 
df['aporte_entrada'] = 0.0
df['aporte_saida'] = 0.0
df['saldo_investido'] = 0.0
df['moedas_compradas'] = 0.0
df['moedas_vendidas'] = 0.0
df['saldo_moedas'] = 0.0
df['preco_medio'] = 0.0
df['saldo_financeiro_real'] = 0.0
df['lucro_atual'] = 0.0
df['saldo_realizado_janela'] = 0.0
df['saldo_realizado'] = 0.0


# Variáveis para controle do saldo
saldo_financeiro = 0
saldo_moedas = 0
saldo_realizado = 0
total_investido = 0
total_moedas_compradas = 0


for index, row in df.iterrows():
    if row['acao'] == 'comprar':
        df.loc[index, 'aporte_entrada'] = APORTE
        saldo_financeiro += APORTE
        df.loc[index, 'moedas_compradas'] = APORTE / row['close']
        saldo_moedas += df.loc[index, 'moedas_compradas']
        
    elif row['acao'] == 'vender': 

       if saldo_financeiro > 0 and row['ultimo_dia_janela'] == False:
            df.loc[index, 'aporte_saida'] = APORTE
            saldo_financeiro -= APORTE
            df.loc[index, 'moedas_vendidas'] = APORTE / row['close']
            saldo_moedas -= df.loc[index, 'moedas_vendidas']
            saldo_realizado += APORTE

       elif (saldo_moedas * row['close']) > 0: 
            # Vender o saldo completo de moedas restantes
            valor_venda = saldo_moedas * row['close']
            df.loc[index, 'aporte_saida'] = valor_venda
            saldo_realizado += valor_venda
            saldo_moedas = 0  # Zerar o saldo de moedas após a venda
            saldo_financeiro = 0  # Zerar o saldo financeiro, pois as moedas foram vendidas


# Atualizar o preço medio, saldo investido, saldo de moedas saldo financeiro real e saldo realizado após cada linha

    df.loc[index, 'saldo_investido'] = saldo_financeiro
    df.loc[index, 'saldo_moedas'] = saldo_moedas
    df.loc[index, 'saldo_financeiro_real'] = saldo_moedas * row['close']
    df.loc[index, 'saldo_realizado'] = saldo_realizado


# Loop para calcular o saldo realizado por janela
janela_atual = df.loc[0, 'janela']
saldo_realizado_janela = 0

for index, row in df.iterrows():
    
    # Verifica se a janela mudou para resetar o saldo realizado da janela
    if row['janela'] != janela_atual:
        janela_atual = row['janela']
        saldo_realizado_janela = 0  # Reinicia o saldo realizado para a nova janela
    

        # Acumula o "aporte_saida" na janela atual
    if row['aporte_saida'] > 0:
        saldo_realizado_janela += row['aporte_saida']
    
    
    # Atualiza o dataframe com o saldo realizado até o momento na janela atual
    df.loc[index, 'saldo_realizado_janela'] = saldo_realizado_janela


    # Calcular o lucro atual como a diferença entre o saldo financeiro real e o saldo investido
    df.loc[index, 'lucro_atual'] = row['saldo_financeiro_real'] - row['saldo_investido']


df

,DATA,close,moeda,mayer,acao,janela,ultimo_dia_janela,aporte_entrada,aporte_saida,saldo_investido,moedas_compradas,moedas_vendidas,saldo_moedas,preco_medio,saldo_financeiro_real,lucro_atual,saldo_realizado_janela,saldo_realizado
1510,2020-02-02,9338.20,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1509,2020-02-03,9300.64,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1508,2020-02-04,9198.35,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1507,2020-02-05,9630.00,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1506,2020-02-06,9778.12,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2024-03-18,67660.00,BTCUSDT,1.666041,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287
3,2024-03-19,61974.10,BTCUSDT,1.519266,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287
2,2024-03-20,67876.40,BTCUSDT,1.655431,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287
1,2024-03-21,65539.90,BTCUSDT,1.590769,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287


In [17]:
# Ordenar por ultima data
df = df.sort_values('DATA', ascending=False)

In [18]:


# Seleciona as linhas do índice 1 ao 30 (desconsidera o índice 0 que é o item mais recente)
Registros_10_head = df.iloc[0:10]  

Registros_10_head

,DATA,close,moeda,mayer,acao,janela,ultimo_dia_janela,aporte_entrada,aporte_saida,saldo_investido,moedas_compradas,moedas_vendidas,saldo_moedas,preco_medio,saldo_financeiro_real,lucro_atual,saldo_realizado_janela,saldo_realizado
0,2024-03-22,63826.0,BTCUSDT,1.542056,vender,12,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287
1,2024-03-21,65539.9,BTCUSDT,1.590769,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287
2,2024-03-20,67876.4,BTCUSDT,1.655431,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287
3,2024-03-19,61974.1,BTCUSDT,1.519266,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287
4,2024-03-18,67660.0,BTCUSDT,1.666041,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287
5,2024-03-17,68447.9,BTCUSDT,1.694146,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287
6,2024-03-16,65351.0,BTCUSDT,1.625776,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287
7,2024-03-15,69552.2,BTCUSDT,1.738434,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287
8,2024-03-14,71455.5,BTCUSDT,1.795756,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287
9,2024-03-13,73127.6,BTCUSDT,1.848314,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287


In [20]:

#import dataset_atual
base_Atual = pl.read_parquet(r'..\03_output\Mayer_teste_de_estrategia.parquet')


#converte dataset em pandas 
base_Atual = base_Atual.to_pandas()


# Ordenar por ultima data
base_Atual = base_Atual.sort_values('DATA', ascending=False)

# Remove a última linha usando drop()
base_Atual = base_Atual.drop(base_Atual.index[0])


base_Atual



,DATA,close,moeda,mayer,acao,janela,ultimo_dia_janela,aporte_entrada,aporte_saida,saldo_investido,moedas_compradas,moedas_vendidas,saldo_moedas,preco_medio,saldo_financeiro_real,lucro_atual,saldo_realizado_janela,saldo_realizado,__index_level_0__
1,2024-03-21,65539.90,BTCUSDT,1.590769,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
2,2024-03-20,67876.40,BTCUSDT,1.655431,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
3,2024-03-19,61974.10,BTCUSDT,1.519266,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
4,2024-03-18,67660.00,BTCUSDT,1.666041,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
5,2024-03-17,68447.90,BTCUSDT,1.694146,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,2020-02-06,9778.12,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1498.0
1507,2020-02-05,9630.00,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1499.0
1508,2020-02-04,9198.35,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1500.0
1509,2020-02-03,9300.64,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1501.0


In [21]:
base_Atual_Datas = pd.DataFrame(base_Atual['DATA'])
base_Atual_Datas

,DATA
1,2024-03-21
2,2024-03-20
3,2024-03-19
4,2024-03-18
5,2024-03-17
...,...
1506,2020-02-06
1507,2020-02-05
1508,2020-02-04
1509,2020-02-03


In [22]:
Registros_10_head_Datas = pd.DataFrame(Registros_10_head['DATA'])
Registros_10_head_Datas

,DATA
0,2024-03-22
1,2024-03-21
2,2024-03-20
3,2024-03-19
4,2024-03-18
5,2024-03-17
6,2024-03-16
7,2024-03-15
8,2024-03-14
9,2024-03-13


In [23]:
Datas_para_concatenar = pd.merge(base_Atual_Datas,Registros_10_head_Datas,how='right', indicator=True)
Datas_para_concatenar = Datas_para_concatenar.query('_merge == "right_only"')
Datas_para_concatenar

,DATA,_merge
0,2024-03-22,right_only


In [24]:
Datas_para_concatenar = Datas_para_concatenar.drop(columns=['_merge'])
Datas_para_concatenar

,DATA
0,2024-03-22


In [25]:
Datas_para_concatenar_selecionado = pd.merge(Datas_para_concatenar,Registros_10_head)
Datas_para_concatenar_selecionado

,DATA,close,moeda,mayer,acao,janela,ultimo_dia_janela,aporte_entrada,aporte_saida,saldo_investido,moedas_compradas,moedas_vendidas,saldo_moedas,preco_medio,saldo_financeiro_real,lucro_atual,saldo_realizado_janela,saldo_realizado
0,2024-03-22,63826.0,BTCUSDT,1.542056,vender,12,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287


In [26]:
# Concatenar resultados
dataset_concat = pd.concat([base_Atual,Datas_para_concatenar_selecionado], axis=0, ignore_index=())
dataset_concat = dataset_concat.drop_duplicates()

In [27]:
dataset_concat

,DATA,close,moeda,mayer,acao,janela,ultimo_dia_janela,aporte_entrada,aporte_saida,saldo_investido,moedas_compradas,moedas_vendidas,saldo_moedas,preco_medio,saldo_financeiro_real,lucro_atual,saldo_realizado_janela,saldo_realizado,__index_level_0__
1,2024-03-21,65539.90,BTCUSDT,1.590769,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
2,2024-03-20,67876.40,BTCUSDT,1.655431,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
3,2024-03-19,61974.10,BTCUSDT,1.519266,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
4,2024-03-18,67660.00,BTCUSDT,1.666041,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
5,2024-03-17,68447.90,BTCUSDT,1.694146,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,2020-02-05,9630.00,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1499.0
1508,2020-02-04,9198.35,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1500.0
1509,2020-02-03,9300.64,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1501.0
1510,2020-02-02,9338.20,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1502.0


In [28]:
# Ordenar por ultima data
dataset_concat = dataset_concat.sort_values('DATA', ascending=False)
dataset_concat = dataset_concat.reset_index()

In [29]:
dataset_concat

,index,DATA,close,moeda,mayer,acao,janela,ultimo_dia_janela,aporte_entrada,aporte_saida,saldo_investido,moedas_compradas,moedas_vendidas,saldo_moedas,preco_medio,saldo_financeiro_real,lucro_atual,saldo_realizado_janela,saldo_realizado,__index_level_0__
0,0,2024-03-22,63826.00,BTCUSDT,1.542056,vender,12,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
1,1,2024-03-21,65539.90,BTCUSDT,1.590769,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
2,2,2024-03-20,67876.40,BTCUSDT,1.655431,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
3,3,2024-03-19,61974.10,BTCUSDT,1.519266,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
4,4,2024-03-18,67660.00,BTCUSDT,1.666041,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,1506,2020-02-06,9778.12,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1498.0
1507,1507,2020-02-05,9630.00,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1499.0
1508,1508,2020-02-04,9198.35,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1500.0
1509,1509,2020-02-03,9300.64,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1501.0


In [30]:
dataset_concat = dataset_concat.drop(columns=['index'])

In [31]:
dataset_concat

,DATA,close,moeda,mayer,acao,janela,ultimo_dia_janela,aporte_entrada,aporte_saida,saldo_investido,moedas_compradas,moedas_vendidas,saldo_moedas,preco_medio,saldo_financeiro_real,lucro_atual,saldo_realizado_janela,saldo_realizado,__index_level_0__
0,2024-03-22,63826.00,BTCUSDT,1.542056,vender,12,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
1,2024-03-21,65539.90,BTCUSDT,1.590769,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
2,2024-03-20,67876.40,BTCUSDT,1.655431,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
3,2024-03-19,61974.10,BTCUSDT,1.519266,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
4,2024-03-18,67660.00,BTCUSDT,1.666041,vender,12,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,2020-02-06,9778.12,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1498.0
1507,2020-02-05,9630.00,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1499.0
1508,2020-02-04,9198.35,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1500.0
1509,2020-02-03,9300.64,BTCUSDT,NaN,manter,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1501.0


In [32]:
#Converter DataFrame do pandas em DataFrame do polars e ganhar performance no export do output
dataset_concat_pl = pl.DataFrame(dataset_concat)
dataset_concat_pl

DATA,close,moeda,mayer,acao,janela,ultimo_dia_janela,aporte_entrada,aporte_saida,saldo_investido,moedas_compradas,moedas_vendidas,saldo_moedas,preco_medio,saldo_financeiro_real,lucro_atual,saldo_realizado_janela,saldo_realizado,__index_level_0__
datetime[ns],f64,str,f64,str,i64,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-03-22 00:00:00,63826.0,"""BTCUSDT""",1.542056,"""vender""",12,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,null
2024-03-21 00:00:00,65539.9,"""BTCUSDT""",1.590769,"""vender""",12,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,null
2024-03-20 00:00:00,67876.4,"""BTCUSDT""",1.655431,"""vender""",12,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,null
2024-03-19 00:00:00,61974.1,"""BTCUSDT""",1.519266,"""vender""",12,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,null
2024-03-18 00:00:00,67660.0,"""BTCUSDT""",1.666041,"""vender""",12,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6327.388535,63014.584287,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2020-02-06 00:00:00,9778.12,"""BTCUSDT""",null,"""manter""",0,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1498.0
2020-02-05 00:00:00,9630.0,"""BTCUSDT""",null,"""manter""",0,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1499.0
2020-02-04 00:00:00,9198.35,"""BTCUSDT""",null,"""manter""",0,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0


In [21]:
#Exportar em formato Parquet
dataset_concat_pl.write_parquet('Mayer_teste_de_estrategia.parquet')

#Exportar em formato Excel
dataset_concat_pl.write_excel('Mayer_teste_de_estrategia.xlsx')